### Import Required Classes

In [1]:
import torch
import torch.nn as nn
from Architectures.Basic_Sequence_classification import Sequence_Classification

### Prepare Model

In [2]:
class Config:
    def __init__(self,
                vocab_size,
                embed_dim=128,
                num_layers=10,
                num_heads=8,
                ff_dim=512,
                pre_normalization=True,
                max_position_embeddings=128,
                dropout_prob=0.1,
                num_labels=6,
                device = "cuda"):
        self.device = device
        self.vocab_size = vocab_size  # Tokenizer vocab size
        self.embed_dim = embed_dim  # Embedding & input to attention
        self.num_layers = num_layers  # Number of encoder layers
        self.num_heads = num_heads  # Number of heads in Multi-Head Attention
        self.ff_dim = ff_dim  # Feed Forward hidden dimension
        self.pre_normalization = pre_normalization  # LayerNorm before or after attention/FFN
        self.max_length = max_position_embeddings  # Max sequence length
        self.dropout_prob = dropout_prob  # Dropout probability
        self.num_classess = num_labels  # Output classes (for classification)

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
model = Sequence_Classification(config=Config(vocab_size=30522)).to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-4)
loss_fn = nn.CrossEntropyLoss()

In [19]:
model = model.to(device)

In [5]:
model.load_state_dict(torch.load("best_model.pt"))
model.classification_head[3] = nn.Linear(in_features=128, out_features=2)

/tmp/ipykernel_6807/2926407311.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("best_model.pt"))


In [6]:
print(model)

Sequence_Classification(
  (position_embedding): SinusoidalEmbeddingLayer(
    (embedding): Embedding(30522, 128)
    (layer_norm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
  )
  (encoder): TransformerEncoder(
    (layers): ModuleList(
      (0-9): 10 x TransformerEncoderLayer(
        (attn): MultiHeadAttention(
          (query): Linear(in_features=128, out_features=128, bias=True)
          (key): Linear(in_features=128, out_features=128, bias=True)
          (value): Linear(in_features=128, out_features=128, bias=True)
          (output): Linear(in_features=128, out_features=128, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (ff): FeedForward2(
          (fc1): Linear(in_features=128, out_features=512, bias=True)
          (fc2): Linear(in_features=512, out_features=128, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
          (gelu): GELU(approximate='none')
        )
        (norm1): LayerNorm((128,), eps=1e-05, elemen

### Prepare Dataset

In [ ]:
from datasets import load_dataset

ds = load_dataset("hungnm/multilingual-amazon-review-sentiment-processed", token="")

README.md:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

dataset_infos.json:   0%|          | 0.00/4.50k [00:00<?, ?B/s]

(…)-00000-of-00008-a8f46fd8ceb7b04a.parquet:   0%|          | 0.00/41.2M [00:00<?, ?B/s]

(…)-00001-of-00008-450b7657a15d6172.parquet:   0%|          | 0.00/41.2M [00:00<?, ?B/s]

(…)-00002-of-00008-8b2c9c4d5313f7f0.parquet:   0%|          | 0.00/41.1M [00:00<?, ?B/s]

...

(…)-00000-of-00001-2981f710859ff2c8.parquet:   0%|          | 0.00/5.77M [00:00<?, ?B/s]

(…)-00000-of-00001-6e49f9231189a550.parquet:   0%|          | 0.00/5.86M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1325529 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/47490 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/47488 [00:00<?, ? examples/s]

In [9]:
ds_en = ds.filter(lambda example: example['language'] == 'en')

Filter:   0%|          | 0/1325529 [00:00<?, ? examples/s]

Filter:   0%|          | 0/47490 [00:00<?, ? examples/s]

Filter:   0%|          | 0/47488 [00:00<?, ? examples/s]

In [11]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
MAX_LEN = 128

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [12]:
def encode(examples):
    return tokenizer(examples['text'], truncation=True, padding='max_length', max_length=MAX_LEN)

encoded_dataset = ds_en.map(encode)
encoded_dataset

Map:   0%|          | 0/216904 [00:00<?, ? examples/s]

Map:   0%|          | 0/7792 [00:00<?, ? examples/s]

Map:   0%|          | 0/7811 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['stars', 'text', 'language', 'label', 'len', 'valid', 'input_ids', 'attention_mask', 'token_type_ids'],
        num_rows: 216904
    })
    validation: Dataset({
        features: ['stars', 'text', 'language', 'label', 'len', 'valid', 'input_ids', 'attention_mask', 'token_type_ids'],
        num_rows: 7792
    })
    test: Dataset({
        features: ['stars', 'text', 'language', 'label', 'len', 'valid', 'input_ids', 'attention_mask', 'token_type_ids'],
        num_rows: 7811
    })
})

In [11]:
len(encoded_dataset['test']['attention_mask'][0])

128

In [13]:
encoded_dataset

DatasetDict({
    train: Dataset({
        features: ['stars', 'text', 'language', 'label', 'len', 'valid', 'input_ids', 'attention_mask', 'token_type_ids'],
        num_rows: 216904
    })
    validation: Dataset({
        features: ['stars', 'text', 'language', 'label', 'len', 'valid', 'input_ids', 'attention_mask', 'token_type_ids'],
        num_rows: 7792
    })
    test: Dataset({
        features: ['stars', 'text', 'language', 'label', 'len', 'valid', 'input_ids', 'attention_mask', 'token_type_ids'],
        num_rows: 7811
    })
})

In [10]:
from torch.utils.data import DataLoader, TensorDataset, Dataset

In [14]:
class Amazon_Dataset(Dataset):
    def __init__(self, dataset):
        self.data = dataset  # HuggingFace Dataset object

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]  # Access row as dict
        return {
            'input_ids': torch.tensor(item['input_ids']),
            'attention_mask': torch.tensor(item['attention_mask']),
            'labels': torch.tensor(item['label'])
        }

train_dataset = Amazon_Dataset(encoded_dataset['train'])
val_dataset = Amazon_Dataset(encoded_dataset['validation'])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=0)
val_loader = DataLoader(val_dataset, batch_size=32)

In [ ]:
config = dict(
    project_name="Sequence_Classification",
    epochs=500,
    classes=10,
    kernels=[16, 32],
    batch_size=128,
    learning_rate=0.005,
    dataset="MNIST",
    architecture="CNN")

In [15]:
len(train_loader)

6779

In [15]:
len(val_loader)

244

### Train and Validate

In [26]:
model = model.to(device)

In [24]:
from sklearn.metrics import accuracy_score, f1_score

In [ ]:
num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for batch_idx, batch in enumerate(train_loader):
        
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        optimizer.zero_grad()
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits if hasattr(outputs, 'logits') else outputs
        
        loss = loss_fn(logits, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        
        if batch_idx % 100 == 0:
            print(f"Batch {batch_idx} Loss: {loss.item():.4f}")
    
    print(f"Epoch {epoch + 1}, Loss: {total_loss / len(train_loader):.4f}")
    
    # Validation
    model.eval()
    preds, targets = [], []
    with torch.no_grad():
        for batch in val_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            
            logits = model(input_ids, attention_mask)
            predictions = torch.argmax(logits, dim=1)
            
            preds.extend(predictions.cpu().numpy())
            targets.extend(labels.cpu().numpy())
    
    acc = accuracy_score(targets, preds)
    f1 = f1_score(targets, preds, average='macro')
    print(f"Validation Accuracy: {acc:.4f}, F1 Score: {f1:.4f}")


Error in callback <bound method _WandbInit._pre_run_cell_hook of <wandb.sdk.wandb_init._WandbInit object at 0x73c17d538980>> (for pre_run_cell), with arguments args (<ExecutionInfo object at 73c035961490, raw_cell="num_epochs = 10

with wandb.init(project="Sequence.." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell:/run/media/seono/P/Sequence/finetune.ipynb#X31sZmlsZQ%3D%3D>,),kwargs {}:


BrokenPipeError: [Errno 32] Broken pipe

BrokenPipeError: [Errno 32] Broken pipe

Error in callback <bound method _WandbInit._post_run_cell_hook of <wandb.sdk.wandb_init._WandbInit object at 0x73c17d538980>> (for post_run_cell), with arguments args (<ExecutionResult object at 73c0359610a0, execution_count=26 error_before_exec=None error_in_exec=[Errno 32] Broken pipe info=<ExecutionInfo object at 73c035961490, raw_cell="num_epochs = 10

with wandb.init(project="Sequence.." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell:/run/media/seono/P/Sequence/finetune.ipynb#X31sZmlsZQ%3D%3D> result=None>,),kwargs {}:


BrokenPipeError: [Errno 32] Broken pipe

In [30]:
# Save entire model
torch.save(model.state_dict(), 'model_weights.pt')

In [31]:
import torch

# Example dummy input for BERT-like models (batch_size=1, sequence_length=128)
dummy_input_ids = torch.randint(0, 1000, (1, 128)).to(device)
dummy_attention_mask = torch.ones((1, 128)).to(device)

In [34]:
torch.onnx.export(
    model,  # Your model
    (dummy_input_ids, dummy_attention_mask),  # Tuple of inputs
    "sequence_classifier.onnx",  # Output file
    input_names=["input_ids", "attention_mask"],
    output_names=["logits"],
    dynamic_axes={
        'input_ids': {0: 'batch_size', 1: 'sequence_length'},
        'attention_mask': {0: 'batch_size', 1: 'sequence_length'},
        'logits': {0: 'batch_size'}
    },
    opset_version=13,
    export_params=True,
    do_constant_folding=True
)


/run/media/seono/P/Sequence/layers/attention.py:130: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  scores = torch.matmul(q, k.transpose(-2, -1)) / torch.sqrt(torch.tensor(self.head_dim, dtype=torch.float32))


In [33]:
!pip install onnx

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 2.4 MB/s eta 0:00:002.4 MB/s eta 0:00:01

[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
